<a href="https://colab.research.google.com/github/simon-mellergaard/GAI-with-LLMs/blob/main/Project%20codes/Assignment03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3
> *Evaluate HuggingFaceTB/SmolLM-135M-Instruct on the cnn_dailymail dataset used in Chapter 6 in Natural Language Processing with Transformers.*
> 1. *Compare four different experiments each with a different prompting and decoding strategy. Argue for why these are the most promising ones to try. Evaluate the rouge scores of each model on the test set (if compute becomes a problem use a smaller subset of the test set) and compare to the performance of PEGASUS.*

## Setup


In [1]:
# Install packages
!pip install datasets==3.6.0 # To load the cnn dataset
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=bde8fb35fd6bcfc41ed58adab7d0e8f4f3f0f97aa1acdafb6b3f0847d5a00fc1
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [2]:
# Libraries
import evaluate
import os
import torch
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize

# Functions
from transformers import pipeline
from google.colab import userdata
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from huggingface_hub import login as login_hf
from wandb import login as login_wandb
from tqdm import tqdm

In [3]:
# Logging in to Huggingface and wandb
os.environ['HF_TOKEN'] = userdata.get('HF')
os.environ['WANDB_TOKEN'] = userdata.get('wandb')
login_hf(os.environ['HF_TOKEN'])
login_wandb(key = os.environ['WANDB_TOKEN'])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: simonmellergaard (simonmellergaard-aarhus-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
# Setting up the device (GPU). Code is taken from the genaibook-module.
def get_device(cuda_ordinal=None):
    if torch.cuda.is_available():
        return torch.device("cuda", cuda_ordinal)
    if torch.backends.mps.is_available():
        return torch.device("mps")
    return torch.device("cpu")

device = get_device()

## Loading the dataset



In [5]:
# Loading the datasets
from datasets import load_dataset

dataset = load_dataset("ccdv/cnn_dailymail", version="3.0.0")
print(f"Features: {dataset['train'].column_names}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

cnn_dailymail.py: 0.00B [00:00, ?B/s]

The repository for ccdv/cnn_dailymail contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ccdv/cnn_dailymail.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


cnn_stories.tgz:   0%|          | 0.00/159M [00:00<?, ?B/s]

dailymail_stories.tgz:   0%|          | 0.00/376M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Features: ['article', 'highlights', 'id']


In [6]:
# Sampling from the test set
test_sampled = dataset["test"].shuffle(seed=42).select(range(1000)) # modify range. originally 1000

## Test

In [7]:
model_name = "gpt2-xl" # Use gpt2 if run out of memory
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_gpt2 = AutoModelForCausalLM.from_pretrained(model_name).to(device)
sample_text = dataset["train"][1]["article"][:2000]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [9]:
max_length = 512
input_txt = dataset["train"][1]["article"][:2000] + '\n\nTL;DR:\n'
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
torch.manual_seed(42)
output_greedy = model_gpt2.generate(input_ids, max_length=max_length,
                               do_sample=True, top_p=0.95, top_k=10, temperature=0.4)
print(tokenizer.decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay. The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds. The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover. The 26-year-old Bolt has now collected eight gold medals at world championships, equaling the record held by American trio Carl Lewis, Michael Johnson and Allyson Felix, not to mention the small matter of six Olympic titles. The relay triumph followed individual successes in the 100 and 200 meters in the Russian capital. "I'm proud of myself and I'll continue to work to dominate for as long as possible," Bolt said, having previously expressed his intention to carry on until the 2016 Rio Olympics. Victory was never seriou

In [11]:
model_name = "HuggingFaceTB/SmolLM-135M-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
sample_text = dataset["train"][1]["article"][:2000]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [12]:
max_length = 128
input_txt = sample_text + '\n\nTL;DR:\n'
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
torch.manual_seed(42)
output_greedy = model.generate(input_ids, max_length=max_length,
                               do_sample=True, top_p=0.95, top_k=10, temperature=0.4)
print(tokenizer.decode(output_greedy[0]))

Both `max_new_tokens` (=40) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay. The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds. The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover. The 26-year-old Bolt has now collected eight gold medals at world championships, equaling the record held by American trio Carl Lewis, Michael Johnson and Allyson Felix, not to mention the small matter of six Olympic titles. The relay triumph followed individual successes in the 100 and 200 meters in the Russian capital. "I'm proud of myself and I'll continue to work to dominate for as long as possible," Bolt said, having previously expressed his intention to carry on until the 2016 Rio Olympics. Victory was never seriou

In [64]:
max_length = 2
input_txt = "News:\n\n" + sample_text + "\n\nSummary of the news: "
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
torch.manual_seed(42)
output = model.generate(input_ids, max_new_tokens=10,
                               do_sample=True, top_p=0.8, top_k=50, temperature=0.2, no_repeat_ngram_size=3)
print(tokenizer.decode(output[0]))

News:

(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay. The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds. The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover. The 26-year-old Bolt has now collected eight gold medals at world championships, equaling the record held by American trio Carl Lewis, Michael Johnson and Allyson Felix, not to mention the small matter of six Olympic titles. The relay triumph followed individual successes in the 100 and 200 meters in the Russian capital. "I'm proud of myself and I'll continue to work to dominate for as long as possible," Bolt said, having previously expressed his intention to carry on until the 2016 Rio Olympics. Victory was never

## SMolLM

The following [SmolLM-Instruct](https://huggingface.co/HuggingFaceTB/SmolLM-135M-Instruct) model is loaded, which is a decoder-only model:

In [41]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In order to create a summary, a promt must be provided to the model, as well as a decoding strategy which relates to how whether or not the output is sampled or not, how many characters to include, hwo to sample etc. This is experimented with in the following ways:

In [137]:
# Run this for a reference model
# pipe_gpt = pipeline("text-generation", model="gpt2-xl")

Device set to use cuda:0


In [106]:
pipe = pipeline("text-generation", model="HuggingFaceTB/SmolLM-135M-Instruct")

Device set to use cuda:0


In [69]:
## EXPERIMENT 1 ##
query = "News:\n\n" + sample_text + "\n\nShort summary of the news: "
torch.manual_seed(42)
pipe_out = pipe(query,
                max_new_tokens=128,
                clean_up_tokenization_spaces=True,
                do_sample=True,
                top_p=0.8,
                top_k=50,
                temperature=0.2,
                no_repeat_ngram_size=3)
print("\n".join(sent_tokenize(pipe_out[0]['generated_text'][len(query) :])))

Device set to use cuda:0


24-year old Usain, who has been a world champion since 21 years, has been declared the world's fastest man by the United Nations.
The United States and the Soviet Union have been competing in the Olympics since 1988, and the two countries have been at the forefront of the world wrestling scene.
The two teams have been in a close race since 30 years, and their combined strength has been the subject of much speculation.
The US team has been in the top 15 in the World Championships since 8 years, while the Soviet team has dominated the 33rd and


In [122]:
## EXPERIMENT 2 ##
query = sample_text + "\n\nTo sum up: "
torch.manual_seed(42)
pipe_out = pipe(query,
                max_new_tokens=128,
                clean_up_tokenization_spaces=True,
                do_sample=True,
                top_p=0.9,
                top_k=100,
                temperature=0.2,
                no_repeat_ngram_size=3
                )
print("\n".join(sent_tokenize(pipe_out[0]['generated_text'][len(query) :])))

1.
The United States has won the 300-meter gold medal in the women's 1992 Olympics.
2.
The US has won 12 gold medals in the Olympic Games.
3.
The USA has won three gold medals.
4.
The Americans have won 2 golds in the Olympics.


In [154]:
## ECPERIMENT 3 ##
query = sample_text + "\nTL;DR: "
# torch.manual_seed(42)
pipe_out = pipe(query,
                max_new_tokens=128,
                clean_up_tokenization_spaces=True,
                do_sample=True,
                top_p=0.9,
                top_k=50,
                temperature=0.2,
                no_repeat_ngram_size=4
                )
print("\n".join(sent_tokenize(pipe_out[0]['generated_text'][len(query) :])))

10-year-olds are being given a chance to prove themselves in the world of sports, with a chance to win a gold medal in the men 4x40m relay, and a chance to be crowned the world's youngest Olympic champion.
**The story:**

The 10th-year-long competition was held in the United States at the University of California, Los Angeles (UCLA).
The event was held in a state-of-the-art arena, with a mix of 10 teams from the United States and 10 from around the world competing in the 4


In [93]:
## EXPERIMENT 4 ##
query = "The text:\n" + sample_text + "\nExtract key points from the text: "
torch.manual_seed(123)
pipe_out = pipe(query,
                max_new_tokens=128,
                clean_up_tokenization_spaces=True,
                do_sample=True,
                top_p=0.9,
                top_k=50,
                temperature=0.1,
                no_repeat_ngram_size=3)
print("\n".join(sent_tokenize(pipe_out[0]['generated_text'][len(query) :])))

1.
The United States was the first team to win the gold medal in the women's 150m sprint, which was held in 1996.
2.
The Jamaican team was the second team to reach the gold, winning the gold in 210m and 120m.
3.
The team was able to complete the 300-meter relay in 50 seconds, which is the fastest time in the Olympics.
4.
The Jamaica team was also the first to win a gold medal, winning in 7000 yards.
5.
The


In [ ]:
sample_text = dataset["train"][1]["article"][:2000]
summaries = {}
queries = ['\nTL;DR:\n',
           '\nSummary of the text:']
for query in queries:
    pipe = pipeline("text-generation", model="HuggingFaceTB/SmolLM-135M-Instruct")
    smollm_query = sample_text + query
    pipe_out = pipe(smollm_query, max_length=512, clean_up_tokenization_spaces=True)
    summaries[query] = "\n".join(
        sent_tokenize(pipe_out[0]["generated_text"][len(smollm_query) :]))

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by p

In [ ]:
    pipe = pipeline("summarization", model="HuggingFaceTB/SmolLM-135M-Instruct")

Device set to use cuda:0
The model 'LlamaForCausalLM' is not supported for summarization. Supported models are ['PeftModelForSeq2SeqLM', 'BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'GraniteSpeechForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForC

In [ ]:
smollm_query = 'sumarize the following text: ' + sample_text + '\n Summary: '
pipe_out = pipe(smollm_query, max_length=50, clean_up_tokenization_spaces=True, do_sample=False, num_beams=5, no_repeat_ngram_size=4)
test_output = "\n".join(
    sent_tokenize(pipe_out[0]["summary_text"][len(smollm_query) :]))

Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [ ]:
test_output

"2023-2024 Olympic Games: The United States and the United Kingdom will compete in the 26th and 27th editions of the Olympic Games, respectively.\nThe United States will participate in the 40th edition of the Games, which will be held in Tokyo, Japan.\nThe United Kingdom will participate in both the 21st and 22nd editions of the Olympics, which will take place in Rio de Janeiro, Brazil.\nThe US and the UK will compete in both the men's and women's events, with the United States winning the men's gold medal and the UK winning the women's gold medal.\nThe US will also participate in the women's singles event, while the UK will participate in men's singles and pair events.\nThe US Olympic Committee will also host the 23rd edition of the Olympics in New York City, which is scheduled to take place in September.\nThe United Nations General Assembly will also hold the 24th edition of this year's Olympics, which is expected to be held in New York, New York, from September 20 to September 24.\n

In [ ]:
print("GROUND TRUTH")
print(dataset["train"][1]["highlights"])
print("")

for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])
    print("")

GROUND TRUTH
Usain Bolt wins third gold of world championship .
Anchors Jamaica to 4x100m relay victory .
Eighth gold at the championships for Bolt .
Jamaica double up in women's 4x100m relay .


TL;DR:

The UK is facing an unprecedented problem with the UK's food supply.
The government is proposing a "Food Safety Bill" which would require food manufacturers to ensure that their products are safe to eat.
The UK is facing an unprecedented problem with the UK's food supply.
The government is proposing a "Food Safety Bill" which would require food manufacturers to ensure that their products are safe to eat.
The bill is seen as an opportunity to drive down food prices and improve food safety standards.
The concept of a "Food Safety Bill" is based on the idea that the government is committed to reducing food waste and improving food safety by requiring food manufacturers to test and meet certain standards.
The bill would require food manufacturers to:
* Ensure that their products meet certa

In [ ]:
summaries

{'SmolLM': 'The 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe'}

In [ ]:
# Loading the rouge metric
rouge_metric = evaluate.load("rouge")

In [ ]:
def chunks(list_of_elements, batch_size):
    """Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def evaluate_summaries_smollm(dataset, metric, model, tokenizer,
                               batch_size=8, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]
        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    score = metric.compute()
    return score

In [ ]:
model_ckpt = "HuggingFaceTB/SmolLM-135M-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForCausalLM.from_pretrained(model_ckpt).to(device)
score = evaluate_summaries_smollm(test_sampled, rouge_metric,
                                   model, tokenizer, batch_size=2) # REDUCED BATCH SIZE

  0%|          | 0/500 [00:00<?, ?it/s]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Both `max_new_tokens` (=40) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
  0%|          | 1/500 [00:03<25:59,  3.13s/it]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Both `max_new_tokens` (=40) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
  0%|          | 2/500 [00:06<25:44,  3.10s/it]A decoder-only architec

In [ ]:
rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=["SmolLM"])

,rouge1,rouge2,rougeL,rougeLsum
SmolLM,0.144227,0.082236,0.105634,0.126501


## Comparing with PEGASUS

In [ ]:
pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail")
pipe_out = pipe(sample_text)
summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" .<n>", ".\n")

In [ ]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
score = evaluate_summaries_pegasus(test_sampled, rouge_metric,
                                   model, tokenizer, batch_size=2) # REDUCED BATCH SIZE